In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/studentgradepassorfailprediction/student-mat-pass-or-fail.csv")

In [ ]:
x_s = df.iloc[:,:-1]
y_s = df.iloc[:,-1]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(x_s,y_s,test_size=0.2,shuffle=True)

In [ ]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test) 

#The fit method is calculating the mean and variance of each of the features present in our data. 
#The transform method is transforming all the features using the respective mean and variance.

In [ ]:
model = Sequential()
model.add(Dense(16,activation='selu',input_shape=[29]))
model.add(Dropout(0.25))
model.add(Dense(16,activation='selu'))
model.add(Dropout(0.25))
model.add(Dense(32,activation='selu'))
model.add(Dropout(0.25))
model.add(Dense(32,activation='selu'))
model.add(Dropout(0.25))
model.add(Dense(16,activation='selu'))
model.add(Dropout(0.25))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# class myCallback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self,epoch,logs={}):
#         if(logs.get('val_accuracy')>0.97):
#             self.model.stop_training=True
#             model.save('my_model.h5')

# callback = myCallback()

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='best.h5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[model_checkpoint_callback])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epoch = range(len(acc))

plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.plot(epoch,acc,label='acc')
plt.plot(epoch,val_acc,label='val_acc')

plt.subplot(1,2,2)
plt.plot(epoch,loss,label='loss')
plt.plot(epoch,val_loss,label='val_loss')

plt.show()

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)

y_pred = y_pred.astype(int).ravel()      

# print(np.concatenate((y_pred.reshape(len(y_pred),1), np.array(y_test).reshape(len(y_test),1)),1))

In [ ]:
labels=[0,1]
matrix = confusion_matrix(np.array(y_test), y_pred, labels=labels)
print(matrix)